In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

import warnings 
warnings.filterwarnings('ignore')

my_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [35]:
#CORRER PRIMERO EL NOTEBOOK FEATURES PARA CREAR LOS CSV DE LABELS Y FEATURES

labels = []
features_por_ventana = []

for i in range(0,7):
    labels.append(pd.read_csv('labels_v'+str(i+1)+'.csv'))
    labels[i].drop('Unnamed: 0',axis='columns',inplace=True)
    features_por_ventana.append(pd.read_csv('features_v'+str(i+1)+'.csv'))
    features_por_ventana[i].drop('Unnamed: 0',axis='columns',inplace=True)

In [36]:
labels[0].head()

,device_id,st,sc
0,1826643666390887030,0.015050,259200.000
1,7037174172278258682,0.029014,259200.000
2,3392065368947589877,0.057540,259200.000
3,1228982273563226229,0.126828,259200.000
4,4123059034628125459,0.132510,17800.664


In [37]:
features_por_ventana[0].head()

,device_id,cantidad_de_subastas_en_ventana,cant_clicks_previos_auct,cant_events_previos_auct,cantidad_de_clicks_en_ventana,tiempo_de_click_promedio,region,cantidad_de_instalaciones_en_ventana,wifi,kind,Porc_I-E,instalacion_open,cant_clicks_previos,cantidad_de_eventos_en_ventana,evento_mas_frecuente,aplicacion_mas_usada,tipo_evento,device_os_version,cant_clicks_previos_event
0,41863526108385,35.0,0.0,0.0,0.0,0.0,0,4.0,Sin Wifi,NaN,0.04544,True,0.0,88.0,1.0,210.0,4.017674e+18,0.0,0.0
1,135153013040192,8.0,0.0,0.0,0.0,0.0,0,1.0,Sin Definir,NaN,0.00000,False,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
2,161514654074162,6.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.00000,0,0.0,8.0,23.0,121.0,6.081165e+18,0.0,0.0
3,181891380775191,1.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.00000,0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
4,186034136943920,7.0,0.0,0.0,0.0,0.0,0,1.0,Sin Definir,list view,0.07690,False,0.0,13.0,1.0,210.0,4.017674e+18,0.0,0.0


# Label

In [38]:
labels_b = []
tiempo_maximo = 259200

for i in range(0,7):
    labels[i] = (labels[i].loc[labels[i]['device_id'].isin(features_por_ventana[i]['device_id'])])[['device_id','st','sc']]
    labels[i]['binary'] = np.where(labels[i]['st']>=tiempo_maximo,False,True)
    labels[i].sort_values(by='device_id',inplace=True)
    labels_b.append(labels[i][['binary','st']].to_records(index=False))

In [39]:
labels_b[0]

rec.array([( True, 157228.465866), ( True, 187854.009137),
           ( True,  10366.357746), ..., ( True,  50088.250389),
           ( True,  81157.046477), ( True,   1081.247579)],
          dtype=[('binary', '?'), ('st', '<f8')])

# Se agrega chequeo para que los df_feautures cumplan condiciones en los algoritmos a ser usados

In [40]:
for i in range(0,7):
    if (features_por_ventana[i].shape[0] != labels[i].shape[0]):
        raise AssertionError("Los df_features de cada ventana tienen que tener misma \
                cantidad de dispositivos(mismos device_id)")

# Algoritmo

In [41]:
from sksurv.column import categorical_to_numeric
for i in range(0,7):
    features_por_ventana[i].replace([np.inf, -np.inf],0,inplace=True)
    features_por_ventana[i]['tipo_evento'] = features_por_ventana[i]['tipo_evento'].astype('category')
    features_por_ventana[i]['region'] = features_por_ventana[i]['tipo_evento'].astype('category')
    features_por_ventana[i]['wifi'] = features_por_ventana[i]['wifi'].astype('category')
    features_por_ventana[i]['kind'] = features_por_ventana[i]['kind'].astype('category')
    features_por_ventana[i]['device_os_version'] = features_por_ventana[i]['device_os_version'].astype('category')
    #features_por_ventana[i]['prediccion_ventana_st_0'] = 0
    #features_por_ventana[i]['prediccion_ventana_st_1'] = 0
    #features_por_ventana[i]['prediccion_ventana_st_2'] = 0
    #features_por_ventana[i]['prediccion_ventana_st_3'] = 0
    #features_por_ventana[i]['prediccion_ventana_st_4'] = 0
    #features_por_ventana[i]['prediccion_ventana_st_5'] = 0
    #features_por_ventana[i]['prediccion_ventana_sc_0'] = 0
    #features_por_ventana[i]['prediccion_ventana_sc_1'] = 0
    #features_por_ventana[i]['prediccion_ventana_sc_2'] = 0
    #features_por_ventana[i]['prediccion_ventana_sc_3'] = 0
    #features_por_ventana[i]['prediccion_ventana_sc_4'] = 0
    #features_por_ventana[i]['prediccion_ventana_sc_5'] = 0
    features_por_ventana[i] = categorical_to_numeric(features_por_ventana[i])
    features_por_ventana[i].fillna(0,inplace=True)
    
    

# GradientBoostingRegressor

In [42]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

gradient_boost_st = GradientBoostingRegressor()
gradient_boost_sc = GradientBoostingRegressor()

def root_mean_squared_error(y_test, predicted):
    return (np.sqrt(mean_squared_error(y_test,predicted)))

In [43]:
print(datetime.datetime.now())

2019-06-24 23:55:28.969992


In [44]:
score_st =[]
score_sc =[]
for i in range(0,4):

    x_train = features_por_ventana[i].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns')
    labels_ventana_sorted = labels[i].sort_values(by='device_id',ascending=True)
    y_train_st = labels_ventana_sorted['st']
    y_train_sc = labels_ventana_sorted['sc']
    x_test =  features_por_ventana[i+3].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns')
    labels_sig_ventana_sorted = labels[i+3].sort_values(by='device_id',ascending=True)
    y_test_st = labels_sig_ventana_sorted['st']
    y_test_sc = labels_sig_ventana_sorted['sc']
    
    gradient_boost_st.fit(x_train,y_train_st)
    predicted_gb_st = gradient_boost_st.predict(x_test)
    gradient_boost_sc.fit(x_train,y_train_sc)
    predicted_gb_sc = gradient_boost_sc.predict(x_test)
    
    #features_por_ventana[i+3]['prediccion_ventana_st_'+str(i)] = pd.Series(predicted_gb_st)
    score_st.append(root_mean_squared_error(y_test_st,predicted_gb_st))
    #features_por_ventana[i+3]['prediccion_ventana_sc_'+str(i)] = pd.Series(predicted_gb_sc)
    score_sc.append(root_mean_squared_error(y_test_sc,predicted_gb_sc))
    

In [45]:
print(datetime.datetime.now())

2019-06-25 00:02:36.158375


In [46]:
score_st

[61404.04172783707, 61699.85679180333, 61204.840467398455, 121122.5061590415]

In [47]:
score_sc

[44113.64546111788, 43835.10433884143, 45199.93451130005, 75498.10039692845]

In [16]:
score_st --> sin pred

[61404.259340446464,
 61699.825655825734,
 61204.873785730866,
 60889.323579433345]

In [17]:
score_sc  --> sin pred

[44113.632461691486, 43835.08116565004, 45199.89894781585, 44746.15397767227]

In [53]:
features_por_ventana[4].shape

(400446, 86)

## Prediccion del target

In [70]:
target = pd.read_csv('../Data/target_competencia_ids.csv')

In [89]:
target['device_id'] = (target['ref_hash'].str.split('_').str[0]).astype('int64')

features_por_ventana[4] = pd.merge(features_por_ventana[4],
                                  target[['device_id']].drop_duplicates(),
                                  on='device_id',
                                  how='outer').drop_duplicates()


In [90]:
features_por_ventana[4]['device_id'].shape

(4037,)

In [91]:
target[['device_id']].drop_duplicates().shape

(4037, 1)

In [92]:
from sksurv.column import categorical_to_numeric
i = 4
features_por_ventana[i].replace([np.inf, -np.inf],0,inplace=True)
features_por_ventana[i]['tipo_evento'] = features_por_ventana[i]['tipo_evento'].astype('category')
features_por_ventana[i]['wifi'] = features_por_ventana[i]['wifi'].astype('category')
features_por_ventana[i]['kind'] = features_por_ventana[i]['kind'].astype('category')
features_por_ventana[i]['device_os_version'] = features_por_ventana[i]['device_os_version'].astype('category')
features_por_ventana[i]['prediccion_ventana_st_0'] = 0
features_por_ventana[i]['prediccion_ventana_st_1'] = 0
features_por_ventana[i]['prediccion_ventana_st_2'] = 0
features_por_ventana[i]['prediccion_ventana_st_3'] = 0
features_por_ventana[i]['prediccion_ventana_st_4'] = 0
features_por_ventana[i]['prediccion_ventana_st_5'] = 0
features_por_ventana[i]['prediccion_ventana_sc_0'] = 0
features_por_ventana[i]['prediccion_ventana_sc_1'] = 0
features_por_ventana[i]['prediccion_ventana_sc_2'] = 0
features_por_ventana[i]['prediccion_ventana_sc_3'] = 0
features_por_ventana[i]['prediccion_ventana_sc_4'] = 0
features_por_ventana[i]['prediccion_ventana_sc_5'] = 0
features_por_ventana[i] = categorical_to_numeric(features_por_ventana[i])
features_por_ventana[i].fillna(0,inplace=True)
    
    

In [93]:
st = gradient_boost_st.predict(features_por_ventana[4].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns'))

In [94]:
sc = gradient_boost_sc.predict(features_por_ventana[4].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns'))

### arreglar la creacion de este CSV !!!  
usar df.append posiblemente para la salida

In [96]:
df1 = pd.DataFrame(pd.Series(st))
df1['ref_hash'] =  (features_por_ventana[4].sort_values(by='device_id',ascending=True)['device_id'])
df1.columns=['obj','ref_hash']
df1['ref_hash'] = df1['ref_hash'].astype('object')
df1['ref_hash'] = [str(int(item))+'_st' for item in df1['ref_hash']]

In [97]:
df1.set_index('ref_hash').to_csv('prediction_st.csv')

In [98]:
df2 = pd.DataFrame(pd.Series(sc))
df2['ref_hash'] =  (features_por_ventana[4].sort_values(by='device_id',ascending=True)['device_id'])
df2.columns=['obj','ref_hash']
df2['ref_hash'] = df2['ref_hash'].astype('object')
df2['ref_hash'] = [str(int(item))+'_sc' for item in df2['ref_hash']]

In [99]:
df2.set_index('ref_hash').to_csv('prediction_sc.csv')

In [39]:
#--> revisar
pd.merge(df1,df2,on='ref_hash',how='outer').head()

,obj_x,ref_hash,obj_y
0,204769.377398,40621409780134_st,NaN
1,201740.329985,90072729247980_st,NaN
2,204180.714358,161514654074162_st,NaN
3,203308.704594,168103949904656_st,NaN
4,119949.478938,186034136943920_st,NaN


In [40]:
labels[i].to_csv('prediction.csv')
features_por_ventana[i].to_csv('features_v'+str(i+1)+'.csv')  

In [ ]:
df_res = df_res.append(res)

# Random Forest

In [18]:
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor()

In [19]:
score_st =[]
score_sc =[]
for i in range(0,4):

    x_train = features_por_ventana[i].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns')
    labels_ventana_sorted = labels[i].sort_values(by='device_id',ascending=True)
    y_train_st = labels_ventana_sorted['st']
    y_train_sc = labels_ventana_sorted['sc']
    x_test =  features_por_ventana[i+3].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns')
    labels_sig_ventana_sorted = labels[i+3].sort_values(by='device_id',ascending=True)
    y_test_st = labels_sig_ventana_sorted['st']
    y_test_sc = labels_sig_ventana_sorted['sc']
    
    random_forest.fit(x_train,y_train_st)
    predicted_gb_st = gradient_boost_st.predict(x_test)
    random_forest.fit(x_train,y_train_sc)
    predicted_gb_sc = gradient_boost_sc.predict(x_test)
    
    features_por_ventana[i+3]['prediccion_ventana_st_'+str(i)] = pd.Series(predicted_gb_st)
    score_st.append(root_mean_squared_error(y_test_st,predicted_gb_st))
    features_por_ventana[i+3]['prediccion_ventana_sc_'+str(i)] = pd.Series(predicted_gb_sc)
    score_sc.append(root_mean_squared_error(y_test_sc,predicted_gb_sc))
    

In [20]:
score_st

[61179.407698179995, 61656.06997222614, 61269.235800742565, 60889.323579433345]

In [22]:
score_sc

[43798.09664628964, 43806.092571001216, 45226.97261949941, 44746.15397767227]

# XGBoost

In [25]:
from xgboost import XGBRegressor
xgboost = XGBRegressor()

In [27]:
score_st =[]
score_sc =[]
for i in range(0,4):

    x_train = features_por_ventana[i].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns')
    labels_ventana_sorted = labels[i].sort_values(by='device_id',ascending=True)
    y_train_st = labels_ventana_sorted['st']
    y_train_sc = labels_ventana_sorted['sc']
    x_test =  features_por_ventana[i+3].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns')
    labels_sig_ventana_sorted = labels[i+3].sort_values(by='device_id',ascending=True)
    y_test_st = labels_sig_ventana_sorted['st']
    y_test_sc = labels_sig_ventana_sorted['sc']
    
    xgboost.fit(x_train,y_train_st)
    predicted_gb_st = gradient_boost_st.predict(x_test)
    xgboost.fit(x_train,y_train_sc)
    predicted_gb_sc = gradient_boost_sc.predict(x_test)
    
    #features_por_ventana[i+3]['prediccion_ventana_st_'+str(i)] = pd.Series(predicted_gb_st)
    score_st.append(root_mean_squared_error(y_test_st,predicted_gb_st))
    #features_por_ventana[i+3]['prediccion_ventana_sc_'+str(i)] = pd.Series(predicted_gb_sc)
    score_sc.append(root_mean_squared_error(y_test_sc,predicted_gb_sc))
    

[23:39:12] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:40:08] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:41:02] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:41:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:42:50] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:42] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:44:34] WARNI

In [30]:
score_st

[61179.407698179995, 61656.06997222614, 61269.235800742565, 60889.323579433345]

In [31]:
score_sc

[43798.09664628964, 43806.092571001216, 45226.97261949941, 44746.15397767227]

# Mayority Voting

In [33]:
from sklearn.ensemble import VotingRegressor
from sklearn import model_selection
#Jugando con cuales se usan se obtienen distintos resultados
estimators = []
estimators.append(('RandomForest',RandomForestRegressor()))
#estimators.append(('DecisionTree', DecisionTreeRegressor()))
estimators.append(('AdaBoost', AdaBoostRegressor()))
#estimators.append(('LinearRegression',LinearRegression()))
estimators.append(('XGBoost',XGBRegressor()))
estimators.append(('GradientBoosting',GradientBoostingRegressor()))
voting = VotingRegressor(estimators)

ImportError: cannot import name 'VotingRegressor' from 'sklearn.ensemble' (C:\Users\Alejandro\Anaconda3\lib\site-packages\sklearn\ensemble\__init__.py)

In [ ]:
score_st =[]
score_sc =[]
for i in range(0,4):

    x_train = features_por_ventana[i].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns')
    labels_ventana_sorted = labels[i].sort_values(by='device_id',ascending=True)
    y_train_st = labels_ventana_sorted['st']
    y_train_sc = labels_ventana_sorted['sc']
    x_test =  features_por_ventana[i+3].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns')
    labels_sig_ventana_sorted = labels[i+3].sort_values(by='device_id',ascending=True)
    y_test_st = labels_sig_ventana_sorted['st']
    y_test_sc = labels_sig_ventana_sorted['sc']
    
    voting.fit(x_train,y_train_st)
    predicted_gb_st = gradient_boost_st.predict(x_test)
    voting.fit(x_train,y_train_sc)
    predicted_gb_sc = gradient_boost_sc.predict(x_test)
    
    #features_por_ventana[i+3]['prediccion_ventana_st_'+str(i)] = pd.Series(predicted_gb_st)
    score_st.append(root_mean_squared_error(y_test_st,xgboost))
    #features_por_ventana[i+3]['prediccion_ventana_sc_'+str(i)] = pd.Series(predicted_gb_sc)
    score_sc.append(root_mean_squared_error(y_test_sc,xgboost))
    

## Algunas pruebas de algoritmos 

In [12]:
from sksurv.preprocessing import OneHotEncoder

features_v1 = OneHotEncoder().fit_transform(features_por_ventana[0])
features_v2 = OneHotEncoder().fit_transform(features_por_ventana[1])
features_v1.head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version,prediccion_ventana_0,prediccion_ventana_1,prediccion_ventana_2,prediccion_ventana_3,prediccion_ventana_4,prediccion_ventana_5
0,41863526108385,35.0,0.0,0.0,0.0,0.0,1.0,34.0,0.0,0.0,...,24.0,0.0,0.0,0,0,0,0,0,0,0
1,135153013040192,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,161514654074162,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,181891380775191,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,186034136943920,7.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


# GradientBoostingSurvivalAnalysis

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

score = []
for i in range(0,4):
    model = GradientBoostingSurvivalAnalysis()
    model.fit(features_por_ventana[i],labels_b[i])
    #prediccion = model.predict(features_por_ventana[i+1])
    #features_por_ventana[i+1]['prediccion_ventana_'+str(i)] = pd.Series(prediccion)
    score.append( model.score(features_por_ventana[i+1], labels_b[i+1]) )

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

model = GradientBoostingSurvivalAnalysis()

In [ ]:
model.fit(features_v1.head(5000),labels_b[0][:5000])

In [ ]:
model.score(features_v2.head(5000),labels_b[1][:5000])

In [ ]:
from sksurv.preprocessing import OneHotEncoder
for i in range(0,5):
    features_por_ventana[i] = OneHotEncoder().fit_transform(features_por_ventana[i])

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

score = []
for i in range(0,1):
    model = GradientBoostingSurvivalAnalysis()
    model.fit(features_por_ventana[i].head(5000),labels_b[i][:5000])
    prediccion = model.predict(features_por_ventana[i+1].head(5000))
    features_por_ventana[i+1]['prediccion_ventana_'+str(i)] = prediccion
    score.append( model.score(features_por_ventana[i+1].head(5000), labels_b[i+1][:5000]) )
    
    
    

In [ ]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

estimator = CoxPHSurvivalAnalysis(alpha=0.0001)
estimator.fit(features_v1.head(10000),labels_b[0][:10000])

In [ ]:
pd.Series(estimator.coef_, index=features_v1.columns)

#### El resultado es un vector de coeficientes, uno para cada variable, donde cada valor corresponde a la relación del registro y evento

In [ ]:
estimator.score(features_v2.head(10000), labels_b[1][:10000])

# Datasets split

In [13]:
from sklearn.metrics import mean_squared_error

def root_mean_squared_error(y_test, predicted):
    return (np.sqrt(mean_squared_error(y_test,predicted)))

In [14]:
df = labels[0].sort_values(ascending=True,by='device_id').head(10000)
df.head()

,device_id,st,binary
257007,41863526108385,157228.465866,True
285766,135153013040192,187854.009137,True
70944,161514654074162,10366.357746,True
316713,181891380775191,256765.420614,True
155213,186034136943920,60166.331894,True


In [15]:
df_feat = features_v1.sort_values(ascending=True,by='device_id').head(10000)
df_feat.head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version,prediccion_ventana_0,prediccion_ventana_1,prediccion_ventana_2,prediccion_ventana_3,prediccion_ventana_4,prediccion_ventana_5
0,41863526108385,35.0,0.0,0.0,0.0,0.0,1.0,34.0,0.0,0.0,...,24.0,0.0,0.0,0,0,0,0,0,0,0
1,135153013040192,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,161514654074162,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,181891380775191,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,186034136943920,7.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [16]:
from sklearn.model_selection import train_test_split

#TRAIN_TEST_SPLIT
#Split arrays or matrices into random train and test subsets
x_train, x_test, y_train, y_test = train_test_split(df_feat.drop(columns='device_id'),
                                                    df['st'],
                                                    test_size=0.2,
                                                    random_state=43)

# KNN

In [28]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

k=5 #-->Hiperparametro
neigh = KNeighborsRegressor(n_neighbors = k ,    #Cantidad de vecinos a usar --> K hiperparametro.
                            n_jobs=-1,           #Define la cantidad de procesadores a usar, -1 todos los disponibles.
                            weights='distance',  #Se define el peso como la inversa a la distancia, los mas cercanos tienen mayor influencia.
                            algorithm='brute'    #Fuerza bruta para el algorimo, podría ser KDTree o sino automatico.
                           )
#En weights se puede pasar una funcion definida por el usuario--> Segundo hiperparametro posible de KNN.
neigh.fit(x_train,y_train) #Fit the model using X as training data and Y as target values
(root_mean_squared_error(y_test, neigh.predict(x_test)), neigh.score(x_train,y_train),neigh.score(x_test,y_test))

(73539.3725672565, 0.5393407379662112, 0.4116223190865531)

# Gradient Boosting

In [27]:
from sklearn.ensemble import GradientBoostingRegressor
gradient_boost = GradientBoostingRegressor()
gradient_boost.fit(x_train,y_train)
(root_mean_squared_error(y_test, gradient_boost.predict(x_test)), gradient_boost.score(x_train,y_train),gradient_boost.score(x_test,y_test))

(24404.462550766973, 0.9372422830604865, 0.9352030080196484)

### Gradient Boosting - GridSearch

In [63]:
from sklearn.model_selection import GridSearchCV
p_test1 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[100,250,500,750,1000,1250,1500,1750]}

tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test1)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'learning_rate': 0.005, 'n_estimators': 1250}, 0.9335394614324747)

In [65]:
gradient_boost_2 = GradientBoostingRegressor(n_estimators= 1250, learning_rate=0.005)
gradient_boost_2.fit(x_train,y_train)
predicted_gb_2 = gradient_boost_2.predict(x_test)
gradient_boost_2.score(x_train, y_train)
root_mean_squared_error(y_test,predicted_gb_2)

24429.581248523762

In [67]:
gradient_boost_2.score(x_test, y_test)
#Baja

0.9350695526174552

In [71]:
p_test2 = {'max_depth':[2,3,4,5,6,7]}
tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test2)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'max_depth': 3}, 0.9338475449027231)

In [72]:
gradient_boost_3 = GradientBoostingRegressor(max_depth= 3)
gradient_boost_3.fit(x_train,y_train)
gradient_boost_3.score(x_test, y_test)
#baja

0.9351380720519836

In [73]:
p_test3 = {'min_samples_split':[2,4,6,8,10,20,40,60,100], 'min_samples_leaf':[1,3,5,7,9] }

tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test3)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'min_samples_leaf': 5, 'min_samples_split': 60}, 0.9339450691357554)

In [74]:
gradient_boost_4 = GradientBoostingRegressor(min_samples_leaf= 5, min_samples_split= 60)
gradient_boost_4.fit(x_train,y_train)
gradient_boost_4.score(x_test, y_test)
#Baja

0.9350079834773051

# Random Forest

In [26]:
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor()
random_forest.fit(x_train,y_train)
(root_mean_squared_error(y_test, random_forest.predict(x_test)), random_forest.score(x_train,y_train),random_forest.score(x_test,y_test))

(26151.213795828015, 0.9637028642597366, 0.9255953544244009)

# Linear Regression

In [25]:
from sklearn.linear_model import LinearRegression
reg_lineal = LinearRegression()
reg_lineal.fit(x_train,y_train)
(root_mean_squared_error(y_test, reg_lineal.predict(x_test)), reg_lineal.score(x_train,y_train),reg_lineal.score(x_test,y_test))

(92393.60424428641, 0.06929584017408585, 0.07124713162299445)

# Ada Boost

In [24]:
from sklearn.ensemble import AdaBoostRegressor
ada_boost = AdaBoostRegressor()
ada_boost.fit(x_train,y_train)
(root_mean_squared_error(y_test, ada_boost.predict(x_test)), ada_boost.score(x_train,y_train),ada_boost.score(x_test,y_test))

(24884.160947039603, 0.9321162772062425, 0.932630650636244)

# Decision Tree

In [51]:
from sklearn.tree import DecisionTreeRegressor
arbol_dec = DecisionTreeRegressor()
arbol_dec.fit(x_train,y_train)
(root_mean_squared_error(y_test, arbol_dec.predict(x_test)), arbol_dec.score(x_train,y_train),arbol_dec.score(x_test,y_test))

### Feature importances

In [57]:
feature_importances = pd.DataFrame(arbol_dec.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances.head(10)

,importance
cantidad_subastas_Thursday,0.778212
cantidad_subastas_Friday,0.173654
cantidad_subastas_Saturday,0.023967
cantidad_de_subastas_en_ventana,0.008015
cantidad_de_eventos_en_ventana,0.001236
aplicacion_mas_usada,0.001119
tipo_evento_mas_frec,0.001060
Porc_I-E,0.000908
cantidad_instalaciones_Thursday,0.000735
cantidad_eventos_Friday,0.000727


# ExtraTreeRegressor

In [23]:
from sklearn.tree import ExtraTreeRegressor
extra_tree = ExtraTreeRegressor()
extra_tree.fit(x_train,y_train)
extra_tree.score(x_test,y_test)

0.8910753425261935

# XGBoost

In [22]:
from xgboost import XGBRegressor
xgboost = XGBRegressor()
xgboost.fit(x_train,y_train)
predicted_xg = xgboost.predict(x_test)
(root_mean_squared_error(y_test,predicted_xg), xgboost.score(x_train,y_train), xgboost.score(x_test,y_test))

[18:56:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(24445.925918910216, 0.9370968849813937, 0.9349826398070178)

### GridSearch

In [ ]:
parameters_xgb = {
        'num_boost_round': [100, 250, 500],
        'eta': [0.05, 0.1, 0.3],
        'max_depth': [6, 9, 12],
        'subsample': [0.9, 1.0],
        'colsample_bytree': [0.9, 1.0],
}

tuning_xgb = GridSearchCV(estimator = XGBRegressor(), param_grid = parameters_xgb)

tuning_xgb.fit(df_feat.drop(columns='device_id'),df['st'])
tuning_xgb.best_params_, tuning.best_score_

# Majority Voting

In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn import model_selection
#Jugando con cuales se usan se obtienen distintos resultados
estimators = []
estimators.append(('RandomForest',RandomForestRegressor()))
#estimators.append(('DecisionTree', DecisionTreeRegressor()))
estimators.append(('AdaBoost', AdaBoostRegressor()))
#estimators.append(('LinearRegression',LinearRegression()))
estimators.append(('XGBoost',XGBRegressor()))
estimators.append(('GradientBoosting',GradientBoostingRegressor()))
voting = VotingRegressor(estimators)
voting.fit(x_train,y_train)
(root_mean_squared_error(y_test, voting.predict(x_test)), voting.score(x_train,y_train),voting.score(x_test,y_test))

# Bagging 

In [20]:
from sklearn.ensemble import BaggingRegressor
bagging = BaggingRegressor()
bagging.fit(x_train,y_train)
(root_mean_squared_error(y_test, bagging.predict(x_test)), bagging.score(x_train,y_train),bagging.score(x_test,y_test))

(26283.321203105806, 0.9637006782608304, 0.9248417196319347)

# HistGradientBoostingRegressor

In [29]:
#Este sirve para grandes cantidades de parametros
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
bagging = HistGradientBoostingRegressor()
bagging.fit(x_train,y_train)
(root_mean_squared_error(y_test, bagging.predict(x_test)), bagging.score(x_train,y_train),bagging.score(x_test,y_test))

KeyboardInterrupt: 

In [100]:
prediction_st = pd.read_csv('prediction_st.csv')

In [101]:
prediction_st.shape

(8074, 2)